In [1]:
import pandas as pd
from pandas import DataFrame

#from functions import split_data, direct_SVM

import numpy as np

import pandas as pd

import sklearn as skl
from sklearn.cross_validation import KFold
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression

from sklearn.externals import joblib

#import matplotlib.pyplot as plt

import numpy as np

#import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
class_label = 1

In [4]:
def split_data(dataset, train_fraction=0.8):
    
    train_size = (train_fraction * np.shape(dataset)[0])
    #test_size = np.shape(dataset)[0] - train_size
    
    np.random.shuffle(dataset)
    
    columns = np.shape(dataset)[1]-1
    x = dataset[0::,0:columns]
    y = dataset[0::,columns:]
    
    x_training, x_test = x[:train_size,:], x[train_size:,:]
    
    y_training, y_test = y[:train_size, :], y[train_size:, :]
    
    return x_training, x_test, y_training, y_test

In [5]:
Michael_prime = pd.read_csv("Trials/Michael.csv", header=0)

Michael_prime.columns = ['Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta',
                         'Low_Gamma', 'High_Gamma', 'Attention', 'Meditation', 'Label']

Michael_prime.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,Label
0,53353,11880,6189,4239,4458,2143,1748,1179,44,78,1
1,99931,29707,46475,15247,7940,9243,2525,2228,48,96,1
2,7455,34486,12475,2774,10723,11459,3134,1632,50,80,1
3,237956,83772,10063,50412,21757,10896,2359,5714,50,77,1
4,68407,9735,2887,2358,4598,2266,897,1531,53,69,1


In [6]:
Michael_data = Michael_prime.values.astype(np.float)

# Michael_data = Michael.values

Michael = Michael_data[0:1000, :]

print(Michael.shape)

(1000, 11)


In [7]:
Mark_prime = pd.read_csv("Trials/Mark.csv", header=0)

Mark_prime.columns = ['Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta',
                         'Low_Gamma', 'High_Gamma', 'Attention', 'Meditation', 'Label']

#class_label += 1

#Mark_data = Mark.sample(400)

Mark_prime.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,Label
0,840783,93268,11747,65774,99985,21266,40141,17742,1,84,2
1,1217192,143461,10210,11761,23137,12823,9761,7286,1,84,2
2,1030498,172887,24678,9847,9626,12134,5483,2068,7,88,2
3,612621,51515,1114,10179,9506,4846,3245,3569,7,75,2
4,2060214,211122,34850,16536,32368,9507,11875,3844,1,60,2


In [8]:
Mark_data = Mark_prime.values.astype(np.float)

# Michael_data = Michael.values

Mark = Mark_data[0:1000, :]

print(Mark.shape)

(1000, 11)


In [9]:
Nathan_prime = pd.read_csv("Trials/Nathan.csv", header=0)

Nathan_prime.columns = ['Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta',
                         'Low_Gamma', 'High_Gamma', 'Attention', 'Meditation', 'Label']

Nathan_prime.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,Label
0,1495717,56904,13756,16496,24743,12446,6140,8738,0,0,3
1,499759,45353,9668,15400,13956,36470,24945,25438,0,0,3
2,704151,323435,36164,80656,51008,30007,9317,9614,0,0,3
3,1355508,167923,67657,93423,106288,312843,101593,95147,0,0,3
4,1501706,198938,20081,119683,52532,28013,11724,15974,0,0,3


In [10]:
Nathan_data = Nathan_prime.values.astype(np.float)

# Michael_data = Michael.values

Nathan = Nathan_data[0:1000, :]

print(Nathan.shape)

(1000, 11)


In [11]:
dataset = np.vstack((Michael, Mark, Nathan))

dataset.shape

(3000, 11)

In [12]:
def eval_SVM(data, test_size=.8, folds=10, this_kernel='linear',
    Cons_range=np.arange(0.5, 16, 0.5), gamma_range=np.arange(0.05, 1, 0.05)):

    columns = np.shape(data)[1]-1

    x = data[0::,0:columns]

    y = data[0::,columns:]

    temp = int(len(data) * test_size)

    kf = KFold(temp, n_folds=folds)
    svc_model = SVC(kernel=this_kernel)
    
    C_range = Cons_range

    Gamma_range = gamma_range

    score_info = []

    for C in C_range:
        for gamma in Gamma_range:
            svc_model.C = C
            svc_model.gamma = gamma
            scores = [svc_model.fit(x[train_indices], y[train_indices]).score(x[test_indices],y[test_indices]) for train_indices, test_indices in kf]
            score = np.mean(scores)
            stuff = (score, C, gamma)
            score_info.append(stuff)



    final_index = np.argmax(score_info, axis=0)

    final_max = score_info[final_index[0]]

    return final_max

In [13]:
def test_SVM(dataset, train_fract=0.8, num_folds=10, kernel_func='linear',
    C_range=np.arange(1, 5, 1), g_range=np.arange(0.1, 0.5, 0.1)):


    (train_acc, C_opt, G_opt) = eval_SVM(dataset, test_size=train_fract, folds=num_folds, this_kernel=kernel_func,
                                         Cons_range=C_range, gamma_range=g_range)

    train_x, test_x, train_y, test_y = split_data(dataset, train_fract) 
    
    svc_model = SVC(kernel=kernel_func, C=C_opt, gamma=G_opt)
    
    svc_model.fit(train_x, train_y)
    
    predicted = svc_model.predict(test_x)
    
    num_correct = 0
    
    for i in range(0, len(predicted)):

        if predicted[i] == test_y[i]:

            num_correct += 1

    accuracy = num_correct/len(predicted)

    return accuracy, test_x, test_y, predicted

In [14]:
SVM_accuracy, test_x, test_y, predicted = test_SVM(dataset, train_fract=0.8, num_folds=5, kernel_func='rbf')

#print(accuracy)

In [15]:
for i in predicted:
    
    print(i)

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0


In [16]:
print(SVM_accuracy)

0.3233333333333333


In [17]:
dfData = pd.DataFrame(test_x)

dfData.columns = ['Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta',
                         'Low_Gamma', 'High_Gamma', 'Attention', 'Meditation']

dfData['True_Label'] = 0

for i in range(len(dfData.axes[0])):
    dfData['True_Label'].iloc[i] = test_y[i]

dfData['SVM'] = 0

classes = np.asarray(predicted)

for i in range(len(dfData.axes[0])):
    dfData['SVM'].iloc[i] = classes[i]

dfData.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,True_Label,SVM
0,191073,61950,7753,18529,5168,11149,4847,7865,47,41,1,2
1,501786,541283,23473,42138,87699,108819,105035,28451,75,53,2,2
2,1414928,10292,16532,10595,6681,5185,4237,1736,69,56,2,2
3,4875,20786,7130,11056,14992,12020,3371,3326,43,60,1,2
4,1764559,1261884,590128,236087,100727,93369,125158,187479,40,44,3,2


In [18]:
def eval_KNN(data, test_size=.8, folds=10, N_range=np.arange(1, 10, 1)):

    columns = np.shape(data)[1]-1

    x = data[0::,0:columns]

    y = data[0::,columns:]

    temp = int(len(data) * test_size)

    kf = KFold(temp, n_folds=folds)
    
    knn_model = KNeighborsClassifier()

    score_info = []

    for n in N_range:
        knn_model.n_neighbors = n
        scores = [knn_model.fit(x[train_indices], y[train_indices]).score(x[test_indices],y[test_indices]) for train_indices, test_indices in kf]
        score = np.mean(scores)
        stuff = (score, n)
        score_info.append(stuff)



    final_index = np.argmax(score_info, axis=0)

    final_max = score_info[final_index[0]]

    return final_max

In [19]:
def test_KNN(dataset, train_fract=0.8, num_folds=10, neigh_range=np.arange(1, 5, 1)):


    (train_acc, N_opt) = eval_KNN(dataset, test_size=train_fract, folds=num_folds, N_range=neigh_range)

    train_x, test_x, train_y, test_y = split_data(dataset, train_fract) 
    
    knn_model = KNeighborsClassifier(n_neighbors=N_opt)
    
    knn_model.fit(train_x, train_y)
    
    predicted = knn_model.predict(test_x)
    
    num_correct = 0
    
    for i in range(0, len(predicted)):

        if predicted[i] == test_y[i]:

            num_correct += 1

    accuracy = num_correct/len(predicted)

    return accuracy, test_x, test_y, predicted, N_opt

In [20]:
Dataset_R2 = dfData.values.astype(np.float)

In [21]:
knn_accuracy, test_x, test_y, predicted, N_opt = test_KNN(dataset, train_fract=0.8, num_folds=5, neigh_range=np.arange(1, 5, 1))

In [22]:
print(knn_accuracy)

0.5933333333333334


In [23]:
dfData['KNN'] = 0

for i in range(len(dfData.axes[0])):
    dfData['KNN'].iloc[i] = test_y[i]

dfData.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,True_Label,SVM,KNN
0,191073,61950,7753,18529,5168,11149,4847,7865,47,41,1,2,2
1,501786,541283,23473,42138,87699,108819,105035,28451,75,53,2,2,1
2,1414928,10292,16532,10595,6681,5185,4237,1736,69,56,2,2,3
3,4875,20786,7130,11056,14992,12020,3371,3326,43,60,1,2,2
4,1764559,1261884,590128,236087,100727,93369,125158,187479,40,44,3,2,1


In [24]:
GNB_model = GaussianNB()

train_x, test_x, train_y, test_y = split_data(dataset, 0.8)

GNB_model.fit(train_x, train_y)

predicted = GNB_model.predict(test_x)
    
num_correct = 0
    
for i in range(0, len(predicted)):

    if predicted[i] == test_y[i]:

        num_correct += 1

GNB_accuracy = num_correct/len(predicted)

print(GNB_accuracy)

0.4583333333333333


In [25]:
dfData['GNB'] = 0

for i in range(len(dfData.axes[0])):
    dfData['GNB'].iloc[i] = test_y[i]

dfData.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,True_Label,SVM,KNN,GNB
0,191073,61950,7753,18529,5168,11149,4847,7865,47,41,1,2,2,3
1,501786,541283,23473,42138,87699,108819,105035,28451,75,53,2,2,1,3
2,1414928,10292,16532,10595,6681,5185,4237,1736,69,56,2,2,3,1
3,4875,20786,7130,11056,14992,12020,3371,3326,43,60,1,2,2,3
4,1764559,1261884,590128,236087,100727,93369,125158,187479,40,44,3,2,1,1


In [26]:
dfData = dfData[['Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'High_Beta',
                         'Low_Gamma', 'High_Gamma', 'Attention', 'Meditation', 'SVM', 'KNN', 'GNB', 'True_Label']]

dfData.head()

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,High_Beta,Low_Gamma,High_Gamma,Attention,Meditation,SVM,KNN,GNB,True_Label
0,191073,61950,7753,18529,5168,11149,4847,7865,47,41,2,2,3,1
1,501786,541283,23473,42138,87699,108819,105035,28451,75,53,2,1,3,2
2,1414928,10292,16532,10595,6681,5185,4237,1736,69,56,2,3,1,2
3,4875,20786,7130,11056,14992,12020,3371,3326,43,60,2,2,3,1
4,1764559,1261884,590128,236087,100727,93369,125158,187479,40,44,2,1,1,3


In [27]:
Dataset_R3 = dfData.values.astype(np.float)

In [28]:
MLR_model = LogisticRegression(solver='newton-cg', multi_class='multinomial')

train_x, test_x, train_y, test_y = split_data(dataset, 0.8)

MLR_model.fit(train_x, train_y)

predicted = MLR_model.predict(test_x)
    
num_correct = 0
    
for i in range(0, len(predicted)):

    if predicted[i] == test_y[i]:

        num_correct += 1

MLR_accuracy = num_correct/len(predicted)

print(MLR_accuracy)

0.5633333333333334


In [29]:
probabilities = MLR_model.predict_proba(test_x)

probabilities

dfProb = pd.DataFrame(probabilities)

dfProb.columns = ['Class_1_prob', 'Class_2_prob', 'Class_3_prob']

dfProb['LR_class'] = 0

for i in range(len(dfProb.axes[0])):
    dfProb['LR_class'].iloc[i] = test_y[i]

dfProb['True_Label'] = 0

for i in range(len(dfProb.axes[0])):
    dfProb['True_Label'].iloc[i] = dfData['True_Label'].iloc[i]
    


dfProb.head()

,Class_1_prob,Class_2_prob,Class_3_prob,LR_class,True_Label
0,0.086490,0.454372,0.459138,3,1
1,0.328143,0.268776,0.403081,3,2
2,0.447592,0.261674,0.290734,1,2
3,0.195218,0.515309,0.289474,2,1
4,0.418387,0.236936,0.344677,1,3


In [30]:
print(SVM_accuracy * 100)

print(knn_accuracy * 100)

print(GNB_accuracy * 100)

print(MLR_accuracy * 100)

32.33333333333333
59.333333333333336
45.83333333333333
56.333333333333336


In [31]:
dfClass = dfProb

dfClass['GNB'] = 0

for i in range(len(dfProb.axes[0])):
    dfClass['GNB'].iloc[i] = dfData['GNB'].iloc[i]


dfClass['SVM'] = dfData['SVM']

dfClass['KNN'] = dfData['KNN']
    
dfClass = dfClass[['Class_1_prob', 'Class_2_prob', 'Class_3_prob', 'LR_class', 'GNB', 'SVM', 'KNN', 'True_Label']]    

dfClass.head()

,Class_1_prob,Class_2_prob,Class_3_prob,LR_class,GNB,SVM,KNN,True_Label
0,0.086490,0.454372,0.459138,3,3,2,2,1
1,0.328143,0.268776,0.403081,3,3,2,1,2
2,0.447592,0.261674,0.290734,1,1,2,3,2
3,0.195218,0.515309,0.289474,2,3,2,2,1
4,0.418387,0.236936,0.344677,1,1,2,1,3


In [32]:
dfInt = dfClass[dfClass['LR_class'] ==  dfClass['KNN']]

dfInt2 = dfClass[dfClass['LR_class'] ==  dfClass['GNB']]

dfAgree = dfInt[dfInt['LR_class'] == dfInt['True_Label']]

dfAgree2 = dfInt2[dfInt2['LR_class'] == dfInt2['True_Label']]

dfCorrect = pd.concat((dfAgree, dfAgree2))

dfCorrect.shape

(124, 8)

In [33]:
dfIntB = dfClass[dfClass['LR_class'] ==  dfClass['KNN']]

dfDisagree = dfIntB[dfIntB['LR_class'] != dfIntB['True_Label']]

dfInt2B = dfClass[dfClass['LR_class'] ==  dfClass['GNB']]

dfDisagree2 = dfInt2B[dfInt2B['LR_class'] != dfInt2B['True_Label']]

dfIncorrect = pd.concat((dfDisagree, dfDisagree2))

dfIncorrect.shape

(266, 8)

In [34]:
num1 = dfAgree.shape[0]

num2 = dfDisagree.shape[0]

num1/num2

0.576271186440678

In [35]:
new_knn = KNeighborsClassifier(n_neighbors=N_opt)

new_mlr = LogisticRegression(solver='newton-cg', multi_class='multinomial')

In [36]:
labels = dataset[:, -1]

features = dataset[:, 0:10]

knn_trained = new_knn.fit(features, labels)

mlr_trained = new_mlr.fit(features, labels)

In [37]:
# joblib.dump(new_mlr, 'mlr.pkl') 

# joblib.dump(new_knn, 'knn.pkl') 